In [10]:

import os
import requests
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [1]:
import pandas as pd
merged_data = pd.read_parquet('/Users/soumyasouravbehera/Downloads/final_data_dropped.parquet', engine = 'pyarrow')

In [16]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [38]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = api_key
# Function to create the prompt
def create_prompt(patient_data):
    return f"""
    Generate a detailed markdown summary for the following patient details:
    - Patient ID: {patient_data}
    - Description: {patient_data['DESCRIPTION'].values()}
    - Birthdate: {patient_data['BIRTHDATE'].values()}
    - Deathdate: {patient_data['DEATHDATE'].values()}
    - Allergies: {patient_data['DESCRIPTION_allergy'].values() or "None"}
    - Careplans: {patient_data['DESCRIPTION_careplan'].values() or "None"}
    - Conditions: {patient_data['DESCRIPTION_condition'].values() or "None"}
    - Immunizations: {patient_data['DESCRIPTION_immunization'].values() or "None"}
    - Procedures: {patient_data['DESCRIPTION_procedures'].values() or "None"}
    
    Ensure the output includes:
    1. Clear sections for each category.
    2. Insights on the patient's data, e.g., if any critical details are missing or require attention.
    3. Suggestions for next steps based on their conditions, if applicable.
    """

# Function to get markdown from OpenAI
def generate_markdown_summary(patient_id):
    # Filter the dataset for the patient
    patient_data = merged_data[merged_data['PATIENT'] == patient_id]
    
    if patient_data.empty:
        return f"No data found for patient ID {patient_id}"
    
    # Convert to dictionary for easier access
    patient_data = patient_data.to_dict()
    
    # Create the prompt
    prompt = create_prompt(patient_data)
    
    # Generate the markdown summary using OpenAI
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Use a suitable model
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled in summarizing patient details."},
                {"role": "user", "content": prompt}
            ]
        )
        result =  response.choices[0].message.content
        return display(Markdown(result))
    except Exception as e:
        return f"Error generating markdown: {str(e)}"
    # return patient_data



# Input: Specify the patient ID
patient_id = "bb66df6a-fe47-4be9-804f-e896ab535b99"
markdown_details = generate_markdown_summary(patient_id)

# # Optionally, save the output to a file
# with open(f'patient_{patient_id}_summary.md', 'w') as file:
#     file.write(markdown_details)


# Patient Summary

## Patient Details
- **Patient ID:** `bb66df6a-fe47-4be9-804f-e896ab535b99`
- **Birthdate:** November 16, 1948
- **Deathdate:** Not applicable (Alive)

---

## Encounters
- **Total Encounters:** 15
- **Types of Encounters:**
  - Outpatient Encounter: 12
  - Check-Up: 3
  - Symptom Encounters: 4

### Insights:
The patient has had multiple outpatient encounters over a span of years, with a mixture of routine check-ups and symptom-related visits. 

---

## Allergies
- No recorded allergies.

### Insights:
The lack of documented allergies is beneficial and simplifies management; however, it would be prudent to confirm this during future encounters.

---

## Care Plans
- **Total Care Plans:** 4 (active)
  - Respiratory therapy
  - Recommendation to avoid exercise
  - Deep breathing and coughing exercises

### Insights:
The patient has a focus on respiratory care, likely due to the recent encounters related to bronchitis. Continuous evaluation and monitoring of their respiratory condition may be required.

---

## Conditions
- **Chronic Conditions:**
  - Polyp of colon (identified)
- **Acute Conditions:**
  - Acute viral pharyngitis (disorder)
  - Acute bronchitis (disorder) (noted in multiple encounters)

### Insights:
The patient is managing conditions that require attention, particularly bronchitis, which has recurred. It is advisable to follow up on their respiratory symptoms and consider further evaluation or interventions.

---

## Immunizations
- **Total Immunizations Recorded:** 9
  - Notable vaccinations include:
    - Influenza seasonal injectable (multiple entries)
    - Pneumococcal conjugate PCV 13
    - Pneumococcal polysaccharide vaccine 23 valent

### Insights:
The patient is consistently receiving influenza vaccinations which is good practice for someone in their age group. It is important to ensure they are up to date with all recommended immunizations.

---

## Procedures
- **Total Procedures:** None recorded.

### Insights:
No procedures have been documented, which could indicate that the patient’s care has been managed conservatively or that this information may not have been collected.

---

## Recommendations for Next Steps
1. **Follow-Up on Respiratory Condition:**
   - Given the history of acute bronchitis, schedule a follow-up appointment to evaluate current symptoms and review care plan adherence.

2. **Regular Screening:**
   - Continue monitoring the polyp of the colon with scheduled colonoscopies or examinations as recommended based on age and risk factors.

3. **Immunization Update:**
   - Assess for any overdue vaccinations, especially for pneumonia given the respiratory conditions.

4. **Confirm Allergy Status:**
   - Validate the patient's allergy history at the next visit to ensure comprehensive care.

5. **Educate on Warning Signs:**
   - Provide the patient with information about recognizing severe respiratory symptoms that warrant immediate medical attention.

--- 

This structured summary provides an overview of the patient's health status and can act as a foundation for ongoing management and care planning.

None
